# 실습 2-1: Softmax Classification 만들기 (with.CNN)

## 1. 필요한 모듈 선언하기

In [1]:
import torch 
from torch import nn, optim, cuda
from torch.utils import data
import torch.nn.functional as F
import time
import torchvision
from torchvision import datasets, transforms
import numpy as np

from torch.autograd import Variable

## 2. Device 선언 및 랜덤 시드 고정

### 2-1) device 선언

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu' # GPU 사용가능하면 사용, 없으면 CPU

### 2-2) 랜덤 시드 고정

In [3]:
torch.manual_seed(777)
if device=='cuda':
  torch.cuda.manual_seed_all(777)

## 3. 데이터셋 불러오기¶

### 3-1) 이미지 전처리

In [4]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### 3-2) 데이터셋 불러오기

In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## 4. 모델 선언

### 4-1) class로 Model 만들기 

In [6]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20 , kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(20, 40, kernel_size=5, padding=2)
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320*8, 32*8)
        self.fc2 = nn.Linear(32*8, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        # print(x.size())
        x = x.view(-1,320*8) # flatten the tensor
        
        x = self.fc3(self.fc2(self.fc1(x)))
        
        return F.log_softmax(x,dim=1)
        # F.softmax() + torch.log() = F.log_softmax()
        # dim=1이어야 함.



4-2) Model 선언

In [7]:
model = Net()
model.to(device)

Net(
  (conv1): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2560, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

## 5. 학습 시작

### 5-1) loss 및 optimizer 선언

In [8]:
optimizer = optim.Adam(model.parameters(), lr= 0.001)

### 5-2) train, test 함수 

In [10]:
 # 학습

def train():
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  #
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in testloader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=True).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(testloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


In [11]:

for epoch in range(1, 10):
    train()
    test()

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301540
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.105326
Train Epoch: 1 [2560/50000 (5%)]	Loss: 1.893014
Train Epoch: 1 [3840/50000 (8%)]	Loss: 1.804483
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.829449
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.511289
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.472245
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.620910
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.476577
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.366448
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.513128
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.496622
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.377253
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.415315
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.506033
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.439693
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.264193
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.251308
Train Epoch: 1 [23040/50000 (46%)]	Loss: 1.187360
Train Epoch: 1 [24320/50000 (49%)]	Loss: 1.330283
Train Epoch: 1 

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0177, Accuracy: 6010/10000 (60%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.089407
Train Epoch: 2 [1280/50000 (3%)]	Loss: 1.070590
Train Epoch: 2 [2560/50000 (5%)]	Loss: 1.079614
Train Epoch: 2 [3840/50000 (8%)]	Loss: 1.099728
Train Epoch: 2 [5120/50000 (10%)]	Loss: 1.201968
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.140289
Train Epoch: 2 [7680/50000 (15%)]	Loss: 1.069656
Train Epoch: 2 [8960/50000 (18%)]	Loss: 1.073724
Train Epoch: 2 [10240/50000 (20%)]	Loss: 1.045197
Train Epoch: 2 [11520/50000 (23%)]	Loss: 1.157052
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.014917
Train Epoch: 2 [14080/50000 (28%)]	Loss: 1.144916
Train Epoch: 2 [15360/50000 (31%)]	Loss: 1.142665
Train Epoch: 2 [16640/50000 (33%)]	Loss: 1.007914
Train Epoch: 2 [17920/50000 (36%)]	Loss: 0.938508
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.086786
Train Epoch: 2 [20480/50000 (41%)]	Loss: 0.968298
Train Epoch: 2 [21760/50000 (43%)]	Loss: 1.035373
Train Epoch: 2 [23040/50000 (46%)]	Loss: 0.999626
Trai